# Instructions:

This file processes the results from running batch1d_posterior_estimation.py. The results are included in the data repository -- see README.txt.

# Import packages
Use kernel "ABM_env" -- see README.

In [ ]:
from __future__ import division
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import math
import time
from tqdm import tqdm
import cProfile
import pickle
from scipy.stats import gaussian_kde
from scipy import stats
from scipy import interpolate
import pandas as pd
from math import sqrt
from multiprocessing import Pool
import multiprocessing as mp
import time
import math
import os
from amcmc import ammcmc

from ABM import SEIR_multiple_pops
from run_simulations import simulate_epidemic_1d
from CalibrationMethod1_methods import *

# Analyze results from batch MCMC

After running code batch1d_posterior_estimation.py, results in folder 1D_calibration_results_071123

In [ ]:
#Load results
MCMC_results_folder = "./Data/MCMC_results/One-Pop"
brute_force_posterior_folder = "./Data/Brute_force_posterior_estimation/One-Pop"
brute_force_file_prefix = "/IND"

#Load in mobility values:
parameter_matrix = pd.read_csv('./Data/Test Data/One-parameter case/variable_parameter_values_One-Pop-NEW-COMBINED-TEST.csv', index_col=0).to_numpy() # Combined batch of MCMC
mobilities = parameter_matrix[:,0]
unique_mobilities = np.linspace(0.005,0.025,17)

## Visualize results

Set "sample_id = 513" to reproduce Fig 9.

In [ ]:
sample_id = 513
nburn = 25000
end = 75000

MCMC_file = MCMC_results_folder+"/AMCMC_sample_ind_"+str(sample_id)+".pickle"
data_file = open(MCMC_file, "rb")
sol = pickle.load(data_file)
data_file.close()

mob_value = mobilities[sample_id]
upper_percentile = np.percentile(sol['chain'][nburn:end], 97.5)
lower_percentile = np.percentile(sol['chain'][nburn:end], 2.5)
upper_percentile_50 = np.percentile(sol['chain'][nburn:end], 75)
lower_percentile_50 = np.percentile(sol['chain'][nburn:end], 25)


# ------------------ Histogram plot --------------------
bins = np.linspace(0.005,0.025,20)
counts, bin_edges = np.histogram(sol['chain'][nburn:end], bins = bins)

fig, ax1 = plt.subplots(dpi=300, figsize = (4,3))

ax1.hist(sol['chain'][nburn:end], bins = bins, label = 'MCMC results')

ax1.set_ylim(0,max(counts))
ax1.set_ylabel('Frequency in MCMC chain')

ax1.set_xlabel('Mobility')

plt.axvline(upper_percentile, color = 'red', label = '95% CI')
plt.axvline(lower_percentile, color = 'red')
plt.axvline(upper_percentile_50, color = 'gold', label = '50% CI')
plt.axvline(lower_percentile_50, color = 'gold')

plt.axvline(mob_value, color = 'limegreen', label = 'True mobility', linestyle = '--')

fig.legend(bbox_to_anchor = (1.39,0.7), borderaxespad=0.)
plt.xticks(unique_mobilities[::4])

# plt.savefig('figs_final/1d_MCMC_results/histogram-plot-sample'+str(sample_id)+'.png',bbox_inches='tight')

plt.show()

#--------- Plot chain/trace plot -------------

plt.figure(dpi = 300, figsize = (4,3))
plt.plot(np.arange(0, end), sol['chain'][0:end], linewidth=0.6)
plt.xlim(0, end)
plt.ylabel('Mobility')
plt.xlabel('Iterations')
# plt.savefig('figs_final/1d_MCMC_results/chain-plot-sample'+str(sample_id)+'.png',bbox_inches='tight')
plt.show()

#------ Plot brute force posterior estimation -------


brute_force_file = brute_force_posterior_folder+brute_force_file_prefix+str(sample_id)+"LOG_PROBS.txt"
brute_likelihood = pd.read_csv(brute_force_file, header=None, sep=" ").to_numpy()


brute_likelihood_y = np.exp(brute_likelihood[:,1])
brute_likelihood_x = brute_likelihood[:,0]

#normalize to get posterior in terms of probability density using np.trapz:
integral = np.trapz(brute_likelihood_y, brute_likelihood_x)
brute_posterior = brute_likelihood_y/integral

plt.figure(dpi = 300, figsize = (4,3))
plt.plot(brute_likelihood_x, brute_posterior)
plt.xlabel('Mobility')
plt.ylabel('Probability density')
plt.xticks(unique_mobilities[::4])

# plt.savefig('figs_final/1d_MCMC_results/brute-posterior-plot-sample'+str(sample_id)+'.png',bbox_inches='tight')
plt.show()



## Check ESS values of MCMC results

In [ ]:
# Check ESS values (first pass):

def check_ESS(sample_id):
    MCMC_file = MCMC_results_folder+"/AMCMC_sample_ind_"+str(sample_id)+".pickle"
    data_file = open(MCMC_file, "rb")
    sol = pickle.load(data_file)
    data_file.close()
    
    n = end-nburn
    auto_corr = compute_group_auto_corr(sol['chain'][nburn:end], 200)
    ESS, went_to_zero = compute_effective_sample_size(n,auto_corr[:,0])
    
    print('*')
    
    return [ESS, went_to_zero]

total = 1000
result = []
for i in range(0,total):
    result.append(check_ESS(i))
    
# total = 1000
# num_workers = mp.cpu_count()
# print(num_workers)
# start_time = time.perf_counter()
# with Pool(num_workers) as pool:
#     result = pool.map(check_ESS, range(0,total))
# finish_time = time.perf_counter()
# print("Program finished in {} seconds - using multiprocessing".format(finish_time-start_time))
# print("---")


In [ ]:
# Save ESS results
user_input = input("Do you want to save the data? Type 'yes' or 'no': ")
if user_input.lower() == 'yes': #So that I don't overwrite the data with zeros
    print('Saving...')
    with open(MCMC_results_folder+'/ESS.pickle', 'wb') as handle:
        pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    print('DID NOT SAVE')

In [ ]:
# Load ESS results
data_file = open(MCMC_results_folder+'/ESS.pickle', "rb") # Combined batch of MCMC
result = pickle.load(data_file)
data_file.close()

In [ ]:
#Update ESS results for samples that didn't have auto-correlation go to zero on the first pass:

updated_result = []

for i in tqdm(range(1000)):
    if result[i][1] == True:
        updated_result.append(result[i])
    else:
        MCMC_file = MCMC_results_folder + "/AMCMC_sample_ind_"+str(i)+".pickle"
        data_file = open(MCMC_file, "rb")
        sol = pickle.load(data_file)
        data_file.close()
        
        time.sleep(1)
        n = end-nburn
        auto_corr = compute_group_auto_corr(sol['chain'][nburn:end], 3000)
        ESS, went_to_zero = compute_effective_sample_size(n,auto_corr[:,0])
        updated_result.append([ESS, went_to_zero])

In [ ]:
# Save ESS results
user_input = input("Do you want to save the data? Type 'yes' or 'no': ")
if user_input.lower() == 'yes': #So that I don't overwrite the data with zeros
    print('Saving...')
    with open(MCMC_results_folder+'/refined_ESS.pickle', 'wb') as handle:
        pickle.dump(updated_result, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    print('DID NOT SAVE')

In [ ]:
# Load ESS results
data_file = open(MCMC_results_folder+'/refined_ESS.pickle', "rb") # Combined batch of MCMC
updated_result = pickle.load(data_file)
data_file.close()

## Simulation based calibration
Plotting reproduces Fig. 11.

In [ ]:
nburn = 25000
end = 75000
L = 50 #trunctation point

#--------------Pull results from folders---------------

# # First batch of MCMC
# MCMC_results_folder = "1D_calibration_results_071123/MCMC_results_071123"
# brute_force_posterior_folder = "1D_calibration_results_071123/Brute_force_posterior_estimation_071123"

# # Second batch of MCMC
ranks = []
ESS_too_small = []

for sample_id in tqdm(range(1000)):
    
#     print('-------------------------')
#     print('-------Sample '+str(sample_id)+'-------')
#     print('-------------------------')
    
    #---------- Check MCMC results -----------
    
    MCMC_file = MCMC_results_folder+"/AMCMC_sample_ind_"+str(sample_id)+".pickle"
    data_file = open(MCMC_file, "rb")
    sol = pickle.load(data_file)
    data_file.close()
    
    # Thin and truncate the chain based on effective sample size and chosen L
    chain_thin_trunc = sol['chain'][nburn:end:int((end-nburn)/updated_result[sample_id][0])]
    print(int((end-nburn)/updated_result[sample_id][0]))
    print(chain_thin_trunc.shape)
    chain_thin_trunc = chain_thin_trunc[:L]
    
    mob_value = mobilities[sample_id]
#     upper_percentile = np.percentile(chain_thin_trunc, 97.5)
#     lower_percentile = np.percentile(chain_thin_trunc, 2.5)
#     upper_percentile_50 = np.percentile(chain_thin_trunc, 75)
#     lower_percentile_50 = np.percentile(chain_thin_trunc, 25)
    
    if chain_thin_trunc.shape[0] < L:
        quantile = np.searchsorted(np.sort(chain_thin_trunc.squeeze()),mob_value)/(chain_thin_trunc.shape[0])
        closest_rank = round(quantile*(L))
        ESS_too_small.append(sample_id)
        ranks.append(closest_rank)
    else:
        ranks.append(np.searchsorted(np.sort(chain_thin_trunc.squeeze()),mob_value))


In [ ]:
# Plot rank histogram:
plt.figure(dpi = 200, figsize = (3,3))
bins = np.arange(0,L+2)
bins = np.arange(0,L+2,1)
bins[-1] = L+1
counts, bins = np.histogram(ranks, bins=bins, range=None, density=None, weights=None)
plt.stairs(counts, bins-0.5, fill = True)
plt.xlabel('Mobility rank')
plt.ylabel('Frequency')

# Chi-squared test:
vals = ranks 
n_bins = 51
counts, bins = np.histogram(vals, bins=n_bins, range=None, density=None, weights=None)
print('p-value:', scipy.stats.chisquare(counts).pvalue)

## Posterior predictive check


In [ ]:
# # Load in training data
# data_file = open('./Data/Training Data/One-parameter case/Calibration method 1/new_I_data_One-Pop-Disc.pickle', "rb")
# data = pickle.load(data_file)
# data_file.close()

# #Load in training data mobility and jumping probability values:
# parameter_matrix = pd.read_csv('./Data/Training Data/One-parameter case/Calibration method 1/variable_parameter_values_One-Pop-Disc.csv', index_col=0).to_numpy()
# mobilities = parameter_matrix[:,0]
# jumping_probs = parameter_matrix[:,1]
# random_seeds = parameter_matrix[:,2]

# #Reshape to sort by mobility and jumping prob:
# #based on assumption that data is given in shape (num_of_sims, num_of_time_steps, num_of_subpops)
# #and that within the list of simulations, the mobility is the "outer" index (changes more slowly)
# #and the jumping probability is the "inner" index (changes more quickly)

# unique_mobilities = np.unique(mobilities) 
# unique_jumping_probs = np.unique(jumping_probs)

# num_of_mobilities = unique_mobilities.shape[0]
# num_of_jumping_probs = unique_jumping_probs.shape[0]

# num_of_random_seeds_per_param_set = round(data.shape[0]/(num_of_mobilities*num_of_jumping_probs)) #assuming equal number of random seeds run for each param set
# num_of_time_steps = data.shape[1]
# # num_of_subpops = 2 #assumes 2 sub-populations

# unique_mobilities = np.unique(mobilities) 
# num_of_mobilities = unique_mobilities.shape[0]

# data_sorted = np.zeros((num_of_mobilities, num_of_random_seeds_per_param_set, num_of_time_steps))

# data_raveled = np.ravel(data, order = 'C')
# data = np.reshape(data_raveled, data_sorted.shape)
# data = np.swapaxes(data, 0, 2)


data_file = open('./Data/Test Data/One-parameter case/new_I_data_One-Pop-NEW-COMBINED-TEST.pickle', "rb") # Combined batch of MCMC
data = pickle.load(data_file)
data_file.close()


In [ ]:
#Choose sample to run posterior predictive check on:
# sample_id = 9
# L = 500 #number of posterior samples to truncate down to
# nburn = 25000
# end = 75000
RUN_NAME = 'Posterior_predictive_check_runs'
DIRECTORY = './Data/Posterior_predictive_check_runs'
os.makedirs(DIRECTORY, exist_ok = True)

def ppc(sample_id, L):
    #Load data from this sample:
    sample_data = data[sample_id, :]

    #Load MCMC runs from this sample:
    MCMC_file = MCMC_results_folder+"/AMCMC_sample_ind_"+str(sample_id)+".pickle"
    data_file = open(MCMC_file, "rb")
    sol = pickle.load(data_file)
    data_file.close()

    #Thin and truncate chain based on chosen L and ESS:
    chain_thin_trunc = sol['chain'][nburn:end:int((end-nburn)/updated_result[sample_id][0])]
    chain_thin_trunc = chain_thin_trunc[:L]

    #Run simulations with the posterior samples:
    #============================================================================================================
    #============================================= Set parameters ===============================================
    #============================================================================================================

    #parameters
    m = 1 #number of populations
    centers = np.array([[0.05,0.05]]) #theta, phi or x, y
    spread = np.array([0.1,0.1]) #standard deviation of normal distribution
    pop = np.array([100,100]) #population
    A_1 = 0.0002 #theta or x mobility (azimuth mobility)
    A_2 = 0.0002 #phi or y mobility (inclination mobility)
    R = 1 #radius
    d_IU = 0.005
    E_0 = np.array([0,0]) #fraction of initially exposed
    I_0 = np.array([0.01,0]) #fraction of initially infected
    S_0 = np.array([0.99,1]) #fraction of initially susceptible
    T_E = 11.6 #time from exposure to infectious
    T_E_stdev = 1.9 #standard deviation of exposure time
    T_I = 18.49 #incubation time
    T_I_stdev = 3.71 #standard deviation of infection time
    del_t = 0.1 #time step
    verlet_iter = 300 #number of steps between updating verlet list #CHANGE THIS LATER
    T = 300
    rand_seed = 1
    g = None
    al = None
    jumping_times = np.ones(int(T/del_t)+1)
    jump_prob = 0.1
    spherical = None
    random_seed = 1
    dist = 'Gamma'

    #============================================================================================================
    #=======================Set up population information files as C++ code inputs=========================
    #============================================================================================================

    num_of_sims = L
    num_of_runs = 5 #split up runs to prevent losing all data at once if run fails, and to avoid slowdowns from high C++ code memory usage
    num_of_sims_per_run = int(num_of_sims/num_of_runs)

    if os.path.exists(DIRECTORY) == False:
        os.makedirs(DIRECTORY)
        os.makedirs(DIRECTORY+'/Pop info stored in parts')
        os.makedirs(DIRECTORY+'/Data stored in parts')

    np.random.seed(2)


    #Set mobility values based on parameters pulled from posterior:
    mobilities = chain_thin_trunc.squeeze()
    jump_probs = 0*np.random.rand(num_of_sims) #jumping probabilities are all 0

    pop_info_array = np.zeros((num_of_sims,14)) 

    #Set population sizes:
    pop_info_array[:,0] = 100 #set all sub-populations to 100

    #Set status fractions (one-population scenario):
    pop_info_array[:,1] = 0.99 #set S fraction to 0.99
    pop_info_array[:,2] = 0 #set E to 0
    pop_info_array[:,3] = 0.01 #set I fraction to 0.01

    #Set domains (one-population scenario):
    pop_info_array[:,5] = 0 #set negative x-lim to 0 for first sub-pop
    pop_info_array[:,6] = 0.1 #set negative x-lim to 0 for first sub-pop
    pop_info_array[:,7] = 0 #set negative y-lim to 0 for all sub-pops
    pop_info_array[:,8] = 0.1 #set negative y-lim to 0 for all sub-pops

    #Set sub-simulation IDs:
    pop_info_array[:,9] = np.arange(num_of_sims)

    #Set seeds:
    pop_info_array[:,10] = np.arange(num_of_sims, num_of_sims+num_of_sims)+80000000 #use different random seeds for new test data

    #Set m values (m = num of subpopulations for a given subsim):
    pop_info_array[:,11] = int(1)

    #Set mobilities:
    pop_info_array[:,12] = mobilities

    #Set jumping probabilities
    pop_info_array[:,13] = jump_probs

    # user_input = input("Do you want to save the data? Type 'yes' or 'no': ")
    # if user_input.lower() == 'yes': #So that I don't overwrite the data with zeros
    print('Saving pop info...')
    df = pd.DataFrame(pop_info_array, columns = ['Total population','S','E','I','DR','x_neg_lim','x_pos_lim','y_neg_lim', 'y_pos_lim', 'Sub-simulation', 'Seed', 'm', 'Mobility', 'Jumping probabilities'], dtype=object)
    df.to_csv(DIRECTORY+'/pop-info-FULL.csv', index = False)

    for i in range(num_of_runs):
        to_write = pop_info_array[num_of_sims_per_run*i:num_of_sims_per_run*i+num_of_sims_per_run,:]
        to_write[:, 9] = np.arange(num_of_sims_per_run) 
        df = pd.DataFrame(to_write, columns = ['Total population','S','E','I','DR','x_neg_lim','x_pos_lim','y_neg_lim', 'y_pos_lim', 'Sub-simulation', 'Seed', 'm', 'Mobility', 'Jumping probabilities'], dtype=object)
        df.to_csv(DIRECTORY+'/Pop info stored in parts/pop-info-part'+str(i)+'.csv', index = False)
    print('Done saving pop info.')

    # else:
    #     print('DID NOT SAVE -- CHANGE VARIABLE allow_save TO 1 TO ALLOW SAVING')

    # allow_s0ve=0

    #============================================================================================================
    #================================== Run simulations and save data ===========================================
    #============================================================================================================
    # Run simulations and save data:
    simulate_epidemic_1d(m, centers, spread, pop, A_1, A_2, R, d_IU, E_0, I_0, S_0, T_E, T_E_stdev, 
                      T_I, T_I_stdev, del_t, verlet_iter, T, rand_seed, g, al, jumping_times, jump_prob, 
                      spherical, random_seed, dist, num_of_sims, num_of_runs, RUN_NAME, DIRECTORY)
    
    return sample_data

In [ ]:
del_t = 0.1 #time step
T = 300
time_vec = np.linspace(0,T,int(T/del_t)+1)

test_seeds = np.random.permutation(np.arange(0,1000))
within_95_conf = []
for i in range(0,100):
    print('i = ', i)
    sample_id = test_seeds[i]
    L = 500
    sample_data = ppc(sample_id, L)

    #Load posterior predictive data:
    data_file = open(DIRECTORY+'/new_I_data_'+RUN_NAME+'.pickle', "rb") # Combined batch of MCMC
    predictive_data = pickle.load(data_file)
    data_file.close()

    def rolling_window_accum(data, steps_to_acc):
        I_ABM_ALL=np.zeros_like(data)
        for i in range(steps_to_acc):
            if i==0:
                I_ABM_ALL+=data*1 # Hard copy
            else:
                I_ABM_ALL[:,i:]+=data[:,:-i]
        return I_ABM_ALL

    def rolling_window_accum_1d(data, steps_to_acc):
        I_ABM_ALL=np.zeros_like(data)
        for i in range(steps_to_acc):
            if i==0:
                I_ABM_ALL+=data*1 # Hard copy
            else:
                I_ABM_ALL[i:]+=data[:-i]
        return I_ABM_ALL

    acc_data = rolling_window_accum(predictive_data, 3001)

    #Plot 95% CI
    lower_conf_bound = np.percentile(acc_data, 2.5, axis = 0)
    upper_conf_bound = np.percentile(acc_data, 97.5, axis = 0)
    median = np.percentile(acc_data, 50, axis = 0)

    plt.figure(dpi = 500, figsize = (4, 3))
    plt.plot(time_vec, rolling_window_accum_1d(sample_data, 3001), label = 'Observed data')
    plt.plot(time_vec, upper_conf_bound, label = 'Upper 95% CI bound')
    plt.plot(time_vec, lower_conf_bound, label = 'Lower 95% CI bound')
    # plt.plot(time_vec, median, label = 'Median')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Total infections')
    
    # plt.savefig('figs_final/MCMC_ppc/CI_plot_sample_id_'+str(sample_id)+'.png',bbox_inches='tight')
    
    plt.show()

    sample_data_acc = rolling_window_accum_1d(sample_data, 3001)

    print("Observed data is within CI's:", np.min(np.array((lower_conf_bound<=sample_data_acc) & (sample_data_acc<=upper_conf_bound))))
    within_95_conf.append(np.min(np.array((lower_conf_bound<=sample_data_acc) & (sample_data_acc<=upper_conf_bound))))